# Extracting amounts for HAC penalties and IPPS payments from HCRIS data

### From page 251 of Chapter 40 from the Provider Reimbursement Manual - Part 2

Line 32 (Corresponds to Worksheet E, Part A, line 70.99) -- Enter the HAC reduction adjustment amount: If you responded “N” on Worksheet S-2, Part I, line 40, column 1, do not complete the HAC reduction adjustment in column 2. If you responded “N” on Worksheet S-2, Part I, line 40, column 2, do not complete the HAC reduction adjustment in column 3. Enter in column 2, the sum of lines 19, 28, 30, 30.01, 31, and 31.01, times 1 percent. For cost reporting periods that overlap October 1, 2014, enter zero in column 2. Enter in column 3, the sum of lines 19, 29, 30, 30.01, 31, and 31.01, times 1 percent. Enter in column 4, the sum of columns 2 and 3. Transfer the amount in column 4 to the cost report calculated settlement, Worksheet E, Part A, line 70.99."

* So, the HAC reduction adjustment is the HAC penalty amount. It corresponds to feature ```E_A_HOS_C1_7099``` and its the sum of two columns (2, 3). These two columns are the sums of lines on the HACRP form, most of which correspond to features in the HCRIS data.
* The instructions say to multiply columns 2 and 3 by 1%. CMS actually means to say "times 0.01". To get the full IPPS payment amount, we simply ignore the multiplication step.

### Let's break it down:

**1.** "If you responded “N” on Worksheet S-2, Part I, line 40, column 1, do not complete the HAC reduction adjustment in column 2. If you responded “N” on Worksheet S-2, Part I, line 40, column 2, do not complete the HAC reduction adjustment in column 3."

* Codes needed: ```S2_1_C1_40, S2_1_C2_40```
* What are these features? Page 38 for details on Worksheet S-2, Part I, Line 40: -- "Enter in column 1, “Y” for yes or “N” for no if your hospital is subject to the HAC reduction adjustment for discharges occurring prior to October 1. For cost reporting periods that overlap October 1, 2014, enter “N” in column 1. Enter in column 2, “Y” for yes or “N” for no if your hospital is subject to the HAC reduction adjustment for discharges occurring on or after October 1."
* So, if ```S2_1_C1_40``` is No, then put a zero in column 2. And, if ```S2_1_C2_40``` is No, then put a zero in column 3.
    
**2.** If values for ```S2_1_C1_40``` are Yes, then you'll need to sum lines 19, 28, 30, 30.01, 31, and 31.01. And, if values for ```S2_1_C2_40``` are Yes, then you'll need to sum lines 19, 29, 30, 30.01, 31, and 31.01. 

* Lines 28, 29, 30, 30.01, 31, and 31.01 have corresponding features in HCRIS worksheets (the HCRIS data):

    Line 28 (Corresponds to Worksheet E, Part A, line 70.96 discharges prior to October 1) -- ```E_A_HOS_C1_7096```   
    Line 29 (Corresponds to Worksheet E, Part A, line 70.97 discharges on or after October 1)-- ```E_A_HOS_C1_7097```      
    Line 30 (Corresponds to Worksheet E, Part A, line 70.93) -- ```E_A_HOS_C1_7093```     
    Line 30.01 (Corresponds to Worksheet E, Part A, line 70.90) -- ```E_A_HOS_C1_7090```     
    Line 31 (Corresponds to Worksheet E, Part A, line 70.94) -- ```E_A_HOS_C1_7094```     
    Line 31.01 (Corresponds to Worksheet E, Part A, line 70.91) -- ```E_A_HOS_C1_7091```     


* Line 19 has no corresponding HCRIS data feature. Instead, it is a subtotal that must be calculated:

    * "Line 19 Subtotal -- Enter in columns 2 and 3, the sum of amounts on lines 15, 16, 17, 17.02, and 18. For SCH, if the hospital specific payment amount on line 14, column 1, is greater than the federal specific payment amount on line 13, column 1, enter in columns 2 and 3, the sum of the amounts on lines 15, 16, 17, and 17.02." 
   
    The above mentioned lines (15, 16, 17, 17.02, 18) correspond to HCRIS data features:    
    Line 15 (Corresponds to Worksheet E, Part A, line 49, Column 1): ```E_A_HOS_C1_49```   
    Line 16 (Corresponds to Worksheet E, Part A, line 50, Column 1): ```E_A_HOS_C1_50```   
    Line 17 (Corresponds to Worksheet E, Part A, line 54, Column 1): ```E_A_HOS_C1_54```   
    Line 17.02 (Corresponds to Worksheet E, Part A, line 68, Column 1): ```E_A_HOS_C1_68```   
    Line 18 (Corresponds to Worksheet E, Part A, line 93, Column 1): ```E_A_HOS_C1_93``` 
        
     SCH's are sole community hospitals (those located 35 or more miles from a like hospital). These can be identified using HCRIS data feature ```S2_1_C1_35```, i.e., If (SCH), enter the # of periods.
     
    
### Let's also grab the IPPS interim payment

```E_A_HOS_C1_72```

### Ultimately, we need these 16 HCRIS features:

```S2_1_C1_35```   
```S2_1_C1_40```      
```S2_1_C2_40```     
```E_A_HOS_C1_72```    
```E_A_HOS_C1_7096```     
```E_A_HOS_C1_7097```     
```E_A_HOS_C1_7093```     
```E_A_HOS_C1_7090```     
```E_A_HOS_C1_7094```      
```E_A_HOS_C1_7091```    
```E_A_HOS_C1_7099```    
```E_A_HOS_C1_49```   
```E_A_HOS_C1_50```   
```E_A_HOS_C1_54```   
```E_A_HOS_C1_68```   
```E_A_HOS_C1_93```    

These features correspond to sheets: E and S2
Parts: 1 and A
Columns: 1 and 2
Lines: 35, 40, 49, 50, 54, 68, 72, 93, 7096, 7097, 7093, 7090, 7094, 7091, 7099

# Part 1: Load, filter, and concatenate large HCRIS PUFs.

In [1]:
import pandas as pd
import warnings
from IPython.utils import io
import sys
import numpy as np
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


main_df = 0
yrs = ['2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']

for yr in yrs:
    
    #########################################################
    #################### ALPHA-NUMERIC TABLE ################
    #########################################################
    
    cols = ['RPT_REC_NUM', 'WKSHT_CD', 'LINE_NUM', 'CLMN_NUM', 'ITM_ALPHNMRC_ITM_TXT']

    alpha_df = pd.read_csv('~/Desktop/Rush/HCRIS/HCRIS_PUFs/HOSP10FY' + yr + '/HOSP10_' + yr + '_ALPHA.CSV', sep=',', header=0, 
                           names=cols, index_col=False,
                           converters={'RPT_REC_NUM':str, 
                                       'WKSHT_CD':str, 
                                       'LINE_NUM':str, 
                                       'CLMN_NUM':str, 
                                       'ITM_ALPHNMRC_ITM_TXT':str})


    WKSHT_CD = alpha_df['WKSHT_CD'].tolist()
    alpha_df['WKSHT'] = [x[0] if x[0] == 'E' else x[0:2] for x in WKSHT_CD]
    alpha_df = alpha_df[alpha_df['WKSHT'].isin(['S2', 'E'])]

    WKSHT_CD = alpha_df['WKSHT_CD'].tolist()
    alpha_df['PART'] = [x[-1] for x in WKSHT_CD]
    alpha_df = alpha_df[alpha_df['PART'].isin(['1', '2', 'A'])]

    alpha_df['COLUMN'] = alpha_df['CLMN_NUM'].tolist()
    alpha_df['LINE'] = alpha_df['LINE_NUM'].tolist()

    alpha_df = alpha_df[alpha_df['CLMN_NUM'].isin(['00000', '00100', '00200', '00300', '00400'])]
    alpha_df = alpha_df[alpha_df['LINE_NUM'].isin(['00200', '03500', '04000', '04700', '04800', '04900', '05000', '05400', '05900', '06800', '07200', '09300', '07096', '07097', '07093', '07090', '07094', '07091', '07099'])]

    # for agreement with HCRIS crosswalk
    alpha_df.replace({'LINE':{'00200':'2', '03500':'35', '04000':'40', '04700':'47', '04800':'48', '04900':'49', '05000':'50', '05400':'54', '05900':'59',  '06800':'68',  '07200':'72', '09300':'93',  '07096':'7096', '07097':'7097', 
                              '07093':'7093', '07090':'7090', '07094':'7094', '07091':'7091', '07099':'7099'}},
                     inplace = True)
    
    # for agreement with HCRIS crosswalk
    alpha_df.replace({'COLUMN':{'00000':'0', '00100':'1', '00200':'2', '00300':'3', '00400':'4'}},
                     inplace = True)

    e_df = alpha_df[alpha_df['WKSHT'] == 'E']
    e_df['CODE'] = e_df['WKSHT'] + '_' + e_df['PART'] + '_HOS_C' + e_df['COLUMN'] + '_' + e_df['LINE']

    s_df = alpha_df[alpha_df['WKSHT'] == 'S2']
    s_df['CODE'] = s_df['WKSHT'] + '_' + s_df['PART'] + '_C' + s_df['COLUMN'] + '_' + s_df['LINE']

    alpha_df = pd.concat([e_df, s_df], ignore_index=True)

    alpha_df = alpha_df.filter(items=['RPT_REC_NUM', 'CODE', 'ITM_ALPHNMRC_ITM_TXT'])
    alpha_df.rename(columns={'ITM_ALPHNMRC_ITM_TXT': 'VALUE'}, inplace=True)
    
    
    #########################################################
    ##################### NUMERIC TABLE #####################
    #########################################################
    
    
    cols = ['RPT_REC_NUM', 'WKSHT_CD', 'LINE_NUM', 'CLMN_NUM', 'ITM_VAL_NUM']

    nmrc_df = pd.read_csv('~/Desktop/Rush/HCRIS/HCRIS_PUFs/HOSP10FY' + yr + '/HOSP10_' + yr + '_NMRC.CSV', sep=',', header=0, 
                           names=cols, index_col=False,
                           converters={'RPT_REC_NUM':str, 
                                       'WKSHT_CD':str, 
                                       'LINE_NUM':str, 
                                       'CLMN_NUM':str, 
                                       'ITM_VAL_NUM':np.float64})


    WKSHT_CD = nmrc_df['WKSHT_CD'].tolist()
    nmrc_df['WKSHT'] = [x[0] if x[0] == 'E' else x[0:2] for x in WKSHT_CD]
    nmrc_df = nmrc_df[nmrc_df['WKSHT'].isin(['S2', 'E'])]

    WKSHT_CD = nmrc_df['WKSHT_CD'].tolist()
    nmrc_df['PART'] = [x[-1] for x in WKSHT_CD]
    nmrc_df = nmrc_df[nmrc_df['PART'].isin(['1', '2', 'A'])]

    nmrc_df['COLUMN'] = nmrc_df['CLMN_NUM'].tolist()
    nmrc_df['LINE'] = nmrc_df['LINE_NUM'].tolist()

    nmrc_df = nmrc_df[nmrc_df['COLUMN'].isin(['00000', '00100', '00200', '00300', '00400'])]
    nmrc_df = nmrc_df[nmrc_df['LINE'].isin(['00200','01300', '01400', '03500', '04000', '04700', '04800', '04900', '05000', '05400', '05900', '06800', '07200', '09300', '07096', '07097', '07093', '07090', '07094', '07091', '07099'])]

    # for agreement with HCRIS crosswalk
    nmrc_df.replace({'LINE':{'00200':'2', '03500':'35', '04000':'40', '04700':'47',  '04800':'48',  '04900':'49',  '05000':'50', '05400':'54', '05900':'59', '06800':'68', '07200':'72', '09300':'93', '07096':'7096', '07097':'7097', '07093':'7093', '07090':'7090', '07094':'7094', '07091':'7091', '07099':'7099',
                             }}, inplace = True)

    nmrc_df.replace({'COLUMN':{'00000':'0', '00100':'1', '00200':'2', '00300':'3', '00400':'4'}}, 
                    inplace = True)

    e_df = nmrc_df[nmrc_df['WKSHT'] == 'E']
    e_df['CODE'] = e_df['WKSHT'] + '_' + e_df['PART'] + '_HOS_C' + e_df['COLUMN'] + '_' + e_df['LINE']

    s_df = nmrc_df[nmrc_df['WKSHT'] == 'S2']
    s_df['CODE'] = s_df['WKSHT'] + '_' + s_df['PART'] + '_C' + s_df['COLUMN'] + '_' + s_df['LINE']

    nmrc_df = pd.concat([e_df, s_df], ignore_index=True)

    nmrc_df = nmrc_df.filter(items=['RPT_REC_NUM', 'CODE', 'ITM_VAL_NUM'])
    nmrc_df.rename(columns={'ITM_VAL_NUM': 'VALUE'}, inplace=True)
    
    
    #########################################################
    ################### CONCATENATE DFs #####################
    #########################################################
    
    
    df = pd.concat([alpha_df, nmrc_df], ignore_index=True)
    
    final_df = df.pivot_table(values='VALUE', index=df['RPT_REC_NUM'], columns='CODE', aggfunc='first')

    final_df.reset_index(inplace=True)
    final_df.columns.name = None

    cols1 = ['RPT_REC_NUM', 'S2_1_C2_2', 'S2_1_C1_35', 'S2_1_C1_40', 'S2_1_C2_40', 'E_A_HOS_C1_72', 'E_A_HOS_C1_7096', 
             'E_A_HOS_C1_7097', 'E_A_HOS_C1_7093', 'E_A_HOS_C1_7090', 'E_A_HOS_C1_7094', 'E_A_HOS_C1_7091', 
             'E_A_HOS_C1_49', 'E_A_HOS_C1_50', 'E_A_HOS_C1_54', 'E_A_HOS_C1_59', 'E_A_HOS_C1_68', 'E_A_HOS_C1_93', 
             'E_A_HOS_C1_47', 'E_A_HOS_C1_48', 'E_A_HOS_C1_7099']

    final_df = final_df.filter(items=cols1, axis=1)
    cols2 = list(final_df)
    
    print('Year:', yr, '|', final_df.shape[0], 'rows, ', final_df.shape[1], 'columns')
    
    missing_list = np.setdiff1d(cols1, cols2) # yields the elements in `cols1` that are NOT in `cols2`
    if len(missing_list) > 0:
        
        print(' -- ', len(missing_list), 'features missing from year' + yr)
        print('   -- ', missing_list, '\n')

    if yr == yrs[0]:
        main_df = final_df.copy(deep=True)
    else:
        main_df = pd.concat([main_df, final_df], ignore_index=True)
    
    
##########  Convert negative (erroneous) HCRIS HAC penalty values to positive values ##########
main_df['E_A_HOS_C1_7099'] = np.abs(main_df['E_A_HOS_C1_7099'])

main_df.sort_values(by='E_A_HOS_C1_7099', ascending=True, inplace=True)
main_df.head(10)

Year: 2015 | 6257 rows,  21 columns
Year: 2016 | 6211 rows,  21 columns
Year: 2017 | 6174 rows,  21 columns
Year: 2018 | 6160 rows,  21 columns
Year: 2019 | 6121 rows,  21 columns
Year: 2020 | 6058 rows,  20 columns
 --  1 features missing from year2020
   --  ['E_A_HOS_C1_93'] 

Year: 2021 | 6050 rows,  20 columns
 --  1 features missing from year2021
   --  ['E_A_HOS_C1_93'] 

Year: 2022 | 2855 rows,  20 columns
 --  1 features missing from year2022
   --  ['E_A_HOS_C1_93'] 

Year: 2023 | 3 rows,  10 columns
 --  11 features missing from year2023
   --  ['E_A_HOS_C1_48' 'E_A_HOS_C1_54' 'E_A_HOS_C1_68' 'E_A_HOS_C1_7090'
 'E_A_HOS_C1_7091' 'E_A_HOS_C1_7093' 'E_A_HOS_C1_7094' 'E_A_HOS_C1_7096'
 'E_A_HOS_C1_7099' 'E_A_HOS_C1_93' 'S2_1_C1_35'] 



RPT_REC_NUM S2_1_C2_2 S2_1_C1_35 S2_1_C1_40 S2_1_C2_40 E_A_HOS_C1_72  \
45095      744516        MS        1.0          Y          N      578546.0   
31413      699112        TX        1.0          Y          N       76191.0   
8283       645789        OK        NaN          Y          N      327887.0   
23643      725518        TN        NaN          Y          N        5475.0   
3193       632292        VA        NaN          N          Y        4965.0   
9374       659081        VA        NaN          Y          Y        7048.0   
5019       693159        MS        NaN          N          Y        7405.0   
7824       640426        WI        NaN          N          Y        7418.0   
32089      720601        CA        NaN          N          Y        7486.0   
9192       653184        LA        NaN          Y          N       10039.0   

      E_A_HOS_C1_7096 E_A_HOS_C1_7097 E_A_HOS_C1_7093 E_A_HOS_C1_7090  \
45095           349.0        135398.0             NaN             NaN   
31413           874.0         14695.0             NaN             NaN   
8283              NaN         72394.0             NaN             NaN   
23643             NaN             NaN             NaN             NaN   
3193              NaN             NaN             NaN             NaN   
9374              NaN             NaN             NaN             NaN   
5019              NaN             NaN             NaN             NaN   
7824              NaN             NaN             NaN             NaN   
32089             NaN             NaN             NaN             NaN   
9192           2330.0             NaN             NaN             NaN   

      E_A_HOS_C1_7094 E_A_HOS_C1_7091 E_A_HOS_C1_49 E_A_HOS_C1_50  \
45095          -531.0             NaN      506417.0       36567.0   
31413           -50.0             NaN       57819.0        4454.0   
8283            -53.0             NaN      300956.0       21052.0   
23643             NaN             NaN           NaN           NaN   
3193              NaN             NaN        6979.0         563.0   
9374              NaN             NaN        7428.0         608.0   
5019            -29.0             NaN        9269.0         721.0   
7824              NaN             NaN        9502.0         774.0   
32089             NaN             NaN        9710.0         773.0   
9192              NaN             NaN        8632.0         687.0   

      E_A_HOS_C1_54 E_A_HOS_C1_59 E_A_HOS_C1_68 E_A_HOS_C1_93 E_A_HOS_C1_47  \
45095           NaN      542984.0           NaN           NaN      506417.0   
31413           NaN       62273.0           NaN           NaN       57819.0   
8283            NaN      322008.0           NaN           NaN      300956.0   
23643           NaN           NaN           NaN           NaN           NaN   
3193            NaN        7542.0           NaN           NaN        6979.0   
9374            NaN        8036.0           NaN           NaN        7428.0   
5019            NaN        9990.0           NaN           NaN        9269.0   
7824            NaN       10276.0           NaN           NaN        9502.0   
32089           NaN       10483.0           NaN           NaN        9710.0   
9192            NaN        9319.0           NaN           NaN        8632.0   

      E_A_HOS_C1_48 E_A_HOS_C1_7099  
45095      439158.0            17.0  
31413        5749.0            53.0  
8283       265783.0            59.0  
23643           NaN            63.0  
3193            NaN            75.0  
9374            NaN            80.0  
5019            NaN           100.0  
7824            NaN           103.0  
32089           NaN           105.0  
9192            NaN           116.0

-----

# Part 2. Impute HAC penalty amounts and IPPS payment amounts

### 1. Separate dataframe into sole community hospitals (SCH) and non-SCH's. 

Sole community hospitals are those located 35 or more miles from a like hospital. These can be identified using HCRIS data feature S2_1_C1_35: "If (SCH), enter the # of periods."


In [2]:
# create a dataframe of SCH hospitals
sch_df = main_df[main_df['S2_1_C1_35'] > 0]
print(sch_df.shape[0], 'rows in dataframe of SCH hospitals')

# create a dataframe of non-SCH hospitals
non_sch_df = main_df[main_df['S2_1_C1_35'].isin([np.nan, float('NaN')])]

print(non_sch_df.shape[0], 'rows in dataframe of non-SCH hospitals\n')

if sch_df.shape[0] + non_sch_df.shape[0] == main_df.shape[0]:
    print('Combined size SCH and non-SCH dataframes equals the size of the main dataframe. Good to go.')
else:
    print('Error. Combined size SCH and non-SCH dataframes DOES NOT equal the size of the main dataframe')

3463 rows in dataframe of SCH hospitals
42426 rows in dataframe of non-SCH hospitals

Combined size SCH and non-SCH dataframes equals the size of the main dataframe. Good to go.


### 2. Separate the SCH dataframe into one where E_A_HOS_C1_48 is greater than E_A_HOS_C1_47 and one where E_A_HOS_C1_48 is not greater than E_A_HOS_C1_47

"For SCH, if the hospital specific payment amount on line 14, column 1, (E_A_HOS_C1_48) is greater than the federal specific payment amount on line 13, column 1, (E_A_HOS_C1_47) enter in columns 2 and 3, the sum of the amounts on lines 15, 16, 17, and 17.02. Otherwise, enter in columns 2 and 3, the sum of amounts on lines 15, 16, 17, 17.02, and 18."

In [3]:
tdf = sch_df.copy(deep=True)
cols = ['E_A_HOS_C1_47',
        'E_A_HOS_C1_48']
for c in cols: 
    tdf[c].fillna(0, inplace=True)

sch_df['E_A_HOS_C1_48 - E_A_HOS_C1_47'] = tdf['E_A_HOS_C1_48'] - tdf['E_A_HOS_C1_47']
sch_1 = sch_df[sch_df['E_A_HOS_C1_48 - E_A_HOS_C1_47'] > 0]
sch_2 = sch_df[(sch_df['E_A_HOS_C1_48 - E_A_HOS_C1_47'] <= 0) | (sch_df['E_A_HOS_C1_48 - E_A_HOS_C1_47'].isin([np.nan, float('NaN')]))]

print('Rows and columns in SCH dataframe:', sch_df.shape[0], sch_df.shape[1])
print('Rows and columns in SCH_1 dataframe:', sch_1.shape[0], sch_1.shape[1])
print('Rows and columns in SCH_2 dataframe:', sch_2.shape[0], sch_2.shape[1])

if sch_1.shape[0] + sch_2.shape[0] == sch_df.shape[0]:
    print('Combined size of SCH 1 and SCH 2 dataframes equal the size of the SCH dataframe. Good to go.')
else:
    print('Error. Combined size of SCH 1 and SCH 2 dataframes DOES NOT equal the size of the SCH dataframe.')


Rows and columns in SCH dataframe: 3463 22
Rows and columns in SCH_1 dataframe: 2337 22
Rows and columns in SCH_2 dataframe: 1126 22
Combined size of SCH 1 and SCH 2 dataframes equal the size of the SCH dataframe. Good to go.


### 3. Compute the line 19 subtotal for non-SCH hospitals and the two types of SCH hospitals

In [4]:
# For SCH's where E_A_HOS_C1_48 > E_A_HOS_C1_47
tdf = sch_1.copy(deep=True)
cols = ['E_A_HOS_C1_49',
        'E_A_HOS_C1_50',
        'E_A_HOS_C1_54',
        'E_A_HOS_C1_68']
for c in cols: 
    tdf[c].fillna(0, inplace=True)
sch_1['Line 19 Subtotal'] = tdf['E_A_HOS_C1_49'] + tdf['E_A_HOS_C1_50'] + tdf['E_A_HOS_C1_54'] + tdf['E_A_HOS_C1_68']
del tdf

# For SCH's where E_A_HOS_C1_48 - E_A_HOS_C1_47 <= 0 or where E_A_HOS_C1_48 - E_A_HOS_C1_47 is NaN
tdf = sch_2.copy(deep=True)
cols = ['E_A_HOS_C1_49',
        'E_A_HOS_C1_50',
        'E_A_HOS_C1_54', 
        'E_A_HOS_C1_68', 
        'E_A_HOS_C1_93']
for c in cols: 
    tdf[c].fillna(0, inplace=True)
sch_2['Line 19 Subtotal'] = tdf['E_A_HOS_C1_49'] + tdf['E_A_HOS_C1_50'] + tdf['E_A_HOS_C1_54'] + tdf['E_A_HOS_C1_68'] + tdf['E_A_HOS_C1_93']
del tdf

# For non-SCH's
tdf = non_sch_df.copy(deep=True)
cols = ['E_A_HOS_C1_49', 
        'E_A_HOS_C1_50', 
        'E_A_HOS_C1_54', 
        'E_A_HOS_C1_68',
        'E_A_HOS_C1_93',
       ]
for c in cols: 
    tdf[c].fillna(0, inplace=True)
non_sch_df['Line 19 Subtotal'] = tdf['E_A_HOS_C1_49'] + tdf['E_A_HOS_C1_50'] + tdf['E_A_HOS_C1_54'] + tdf['E_A_HOS_C1_68'] + tdf['E_A_HOS_C1_93']
del tdf


main_df2 = pd.concat([sch_1, sch_2, non_sch_df], ignore_index=True)
main_df2.drop(labels=['E_A_HOS_C1_48 - E_A_HOS_C1_47'], axis=1, inplace=True)

print(main_df2.shape)
main_df2.head()

(45889, 22)


RPT_REC_NUM S2_1_C2_2 S2_1_C1_35 S2_1_C1_40 S2_1_C2_40 E_A_HOS_C1_72  \
0      663472        NV        1.0          Y          N       76603.0   
1      686342        GA        1.0          Y          N     1844958.0   
2      720491        OK        1.0          N          Y     1291634.0   
3      635493        MI        1.0          N          Y     4758560.0   
4      654349        OK        1.0          Y          N      147376.0   

  E_A_HOS_C1_7096 E_A_HOS_C1_7097 E_A_HOS_C1_7093 E_A_HOS_C1_7090  \
0         16597.0             NaN             NaN             NaN   
1        100573.0        252693.0         10195.0             NaN   
2        249410.0         29142.0             NaN             NaN   
3        643860.0        121652.0          1588.0             NaN   
4         33258.0             NaN           298.0             NaN   

  E_A_HOS_C1_7094 E_A_HOS_C1_7091 E_A_HOS_C1_49 E_A_HOS_C1_50 E_A_HOS_C1_54  \
0             NaN             NaN       66388.0           NaN           NaN   
1        -11266.0             NaN     1701595.0      111578.0           NaN   
2         -3125.0             NaN      906755.0       49396.0      158055.0   
3          -607.0             NaN     4204659.0      246188.0        3175.0   
4          -471.0             NaN      124268.0        8765.0           NaN   

  E_A_HOS_C1_59 E_A_HOS_C1_68 E_A_HOS_C1_93 E_A_HOS_C1_47 E_A_HOS_C1_48  \
0       66388.0           NaN           NaN       49600.0       66388.0   
1     1813173.0           NaN           NaN     1410425.0     1701595.0   
2     1114206.0           NaN           NaN      859754.0      906755.0   
3     4454022.0           NaN           NaN     3308013.0     4204659.0   
4      133033.0           NaN           NaN      112224.0      124268.0   

  E_A_HOS_C1_7099  Line 19 Subtotal  
0           830.0           66388.0  
1          1293.0         1813173.0  
2          1457.0         1114206.0  
3          1647.0         4454022.0  
4          1661.0          133033.0

In [5]:
main_df3 = main_df2.copy(deep=True)

cols = ['E_A_HOS_C1_7091', 
        'E_A_HOS_C1_7094', 
        'E_A_HOS_C1_7090', 
        'E_A_HOS_C1_7093', 
        'E_A_HOS_C1_7096',
        'E_A_HOS_C1_7097',
        'Line 19 Subtotal']

for c in cols: 
    main_df3[c] = main_df3[c].astype(float)
    main_df3[c].fillna(0, inplace=True)
    
main_df2['Reconstructed IPPS payment (pre HAC penalty)'] = np.round(main_df3['E_A_HOS_C1_7091'] + main_df3['E_A_HOS_C1_7094'] + main_df3['E_A_HOS_C1_7090'] + main_df3['E_A_HOS_C1_7093'] + main_df3['E_A_HOS_C1_7096'] + main_df3['E_A_HOS_C1_7097'] + main_df3['Line 19 Subtotal'], 0)
main_df2['Reconstructed HAC penalty'] = np.round(0.01 * main_df2['Reconstructed IPPS payment (pre HAC penalty)'], 0)
main_df2['Reconstructed IPPS payment (post HAC penalty)'] = np.round(0.99 * main_df2['Reconstructed IPPS payment (pre HAC penalty)'], 0)

main_df2['HAC penalty imputed from E_A_HOS_C1_59'] = 0.01 * main_df2['E_A_HOS_C1_59']
del main_df3

print(main_df2.shape)
main_df2.head()


(45889, 26)


RPT_REC_NUM S2_1_C2_2 S2_1_C1_35 S2_1_C1_40 S2_1_C2_40 E_A_HOS_C1_72  \
0      663472        NV        1.0          Y          N       76603.0   
1      686342        GA        1.0          Y          N     1844958.0   
2      720491        OK        1.0          N          Y     1291634.0   
3      635493        MI        1.0          N          Y     4758560.0   
4      654349        OK        1.0          Y          N      147376.0   

  E_A_HOS_C1_7096 E_A_HOS_C1_7097 E_A_HOS_C1_7093 E_A_HOS_C1_7090  \
0         16597.0             NaN             NaN             NaN   
1        100573.0        252693.0         10195.0             NaN   
2        249410.0         29142.0             NaN             NaN   
3        643860.0        121652.0          1588.0             NaN   
4         33258.0             NaN           298.0             NaN   

  E_A_HOS_C1_7094 E_A_HOS_C1_7091 E_A_HOS_C1_49 E_A_HOS_C1_50 E_A_HOS_C1_54  \
0             NaN             NaN       66388.0           NaN           NaN   
1        -11266.0             NaN     1701595.0      111578.0           NaN   
2         -3125.0             NaN      906755.0       49396.0      158055.0   
3          -607.0             NaN     4204659.0      246188.0        3175.0   
4          -471.0             NaN      124268.0        8765.0           NaN   

  E_A_HOS_C1_59 E_A_HOS_C1_68 E_A_HOS_C1_93 E_A_HOS_C1_47 E_A_HOS_C1_48  \
0       66388.0           NaN           NaN       49600.0       66388.0   
1     1813173.0           NaN           NaN     1410425.0     1701595.0   
2     1114206.0           NaN           NaN      859754.0      906755.0   
3     4454022.0           NaN           NaN     3308013.0     4204659.0   
4      133033.0           NaN           NaN      112224.0      124268.0   

  E_A_HOS_C1_7099  Line 19 Subtotal  \
0           830.0           66388.0   
1          1293.0         1813173.0   
2          1457.0         1114206.0   
3          1647.0         4454022.0   
4          1661.0          133033.0   

   Reconstructed IPPS payment (pre HAC penalty)  Reconstructed HAC penalty  \
0                                       82985.0                      830.0   
1                                     2165368.0                    21654.0   
2                                     1389633.0                    13896.0   
3                                     5220515.0                    52205.0   
4                                      166118.0                     1661.0   

   Reconstructed IPPS payment (post HAC penalty)  \
0                                        82155.0   
1                                      2143714.0   
2                                      1375737.0   
3                                      5168310.0   
4                                       164457.0   

  HAC penalty imputed from E_A_HOS_C1_59  
0                                 663.88  
1                               18131.73  
2                               11142.06  
3                               44540.22  
4                                1330.33

------

# Comparisons

In [6]:
test_df = main_df2[main_df2['E_A_HOS_C1_7099'] > 0]
test_df = test_df[test_df['Reconstructed HAC penalty'] > 0]
test_df = test_df[test_df['S2_1_C1_35'].isin([0, np.nan, float('NaN')])]
#test_df = test_df[test_df['S2_1_C1_40'] == 'Y']
#test_df = test_df[test_df['S2_1_C2_40'] == 'N']
#test_df = test_df[test_df['FILE_YEAR'].isin(['2020'])]
print('test_df.shape:', test_df.shape)


penalties = test_df['E_A_HOS_C1_7099'].tolist()
T = np.round(np.array(test_df['E_A_HOS_C1_7099'].tolist()),0)
O = np.round(np.array(test_df['Reconstructed HAC penalty'].tolist()),0)

# Get percent error
test_df['% error'] = 100 * np.abs(O - T) / (T)

print('Stats on % error between E_A_HOS_C1_7099 (HAC reduction adjustment amount) and reconstructed HAC penalties:\n')
print('mean % error:', np.nanmean(test_df['% error']))
print('median % error:', np.nanmedian(test_df['% error']))
print('min % error:', np.nanmin(test_df['% error']))
print('max % error:', np.nanmax(test_df['% error']))

print('\nPercent of estimates within 0% error:', np.round(100 * test_df[test_df['% error'] <= 0].shape[0]/test_df.shape[0], 2))
print('Percent of estimates within 1% error:', np.round(100 * test_df[test_df['% error'] <= 1].shape[0]/test_df.shape[0], 2))
print('Percent of estimates within 5% error:', np.round(100 * test_df[test_df['% error'] <= 5].shape[0]/test_df.shape[0], 2))
print('Percent of estimates within 10% error:', np.round(100 * test_df[test_df['% error'] <= 10].shape[0]/test_df.shape[0], 2),'\n')

col_to_move = test_df.pop('E_A_HOS_C1_7099')
test_df.insert(test_df.shape[1]-1, 'E_A_HOS_C1_7099', col_to_move)
col_to_move = test_df.pop('Reconstructed HAC penalty')
test_df.insert(test_df.shape[1]-1, 'Reconstructed HAC penalty', col_to_move)

test_df.sort_values(by=['% error'], inplace=True, ascending=True)

main_df2['Reconstructed HAC penalty'].replace(0, np.nan, inplace=True)
main_df2['Line 19 Subtotal'].replace(0, np.nan, inplace=True)

print("Number of real-valued rows for columns in the final dataframe:")
main_df2.notna().sum()

main_df2.sort_values(by='E_A_HOS_C1_7099', ascending=True, inplace=True)
    
main_df2.head(10)

test_df.shape: (6369, 26)
Stats on % error between E_A_HOS_C1_7099 (HAC reduction adjustment amount) and reconstructed HAC penalties:

mean % error: 154.3324803338417
median % error: 2.167622651004366
min % error: 0.0
max % error: 256811.79698216735

Percent of estimates within 0% error: 38.94
Percent of estimates within 1% error: 49.02
Percent of estimates within 5% error: 51.25
Percent of estimates within 10% error: 52.88 

Number of real-valued rows for columns in the final dataframe:


RPT_REC_NUM S2_1_C2_2 S2_1_C1_35 S2_1_C1_40 S2_1_C2_40 E_A_HOS_C1_72  \
2337      744516        MS        1.0          Y          N      578546.0   
2338      699112        TX        1.0          Y          N       76191.0   
3463      645789        OK        NaN          Y          N      327887.0   
3464      725518        TN        NaN          Y          N        5475.0   
3465      632292        VA        NaN          N          Y        4965.0   
3466      659081        VA        NaN          Y          Y        7048.0   
3467      693159        MS        NaN          N          Y        7405.0   
3468      640426        WI        NaN          N          Y        7418.0   
3469      720601        CA        NaN          N          Y        7486.0   
3470      653184        LA        NaN          Y          N       10039.0   

     E_A_HOS_C1_7096 E_A_HOS_C1_7097 E_A_HOS_C1_7093 E_A_HOS_C1_7090  \
2337           349.0        135398.0             NaN             NaN   
2338           874.0         14695.0             NaN             NaN   
3463             NaN         72394.0             NaN             NaN   
3464             NaN             NaN             NaN             NaN   
3465             NaN             NaN             NaN             NaN   
3466             NaN             NaN             NaN             NaN   
3467             NaN             NaN             NaN             NaN   
3468             NaN             NaN             NaN             NaN   
3469             NaN             NaN             NaN             NaN   
3470          2330.0             NaN             NaN             NaN   

     E_A_HOS_C1_7094 E_A_HOS_C1_7091 E_A_HOS_C1_49 E_A_HOS_C1_50  \
2337          -531.0             NaN      506417.0       36567.0   
2338           -50.0             NaN       57819.0        4454.0   
3463           -53.0             NaN      300956.0       21052.0   
3464             NaN             NaN           NaN           NaN   
3465             NaN             NaN        6979.0         563.0   
3466             NaN             NaN        7428.0         608.0   
3467           -29.0             NaN        9269.0         721.0   
3468             NaN             NaN        9502.0         774.0   
3469             NaN             NaN        9710.0         773.0   
3470             NaN             NaN        8632.0         687.0   

     E_A_HOS_C1_54 E_A_HOS_C1_59 E_A_HOS_C1_68 E_A_HOS_C1_93 E_A_HOS_C1_47  \
2337           NaN      542984.0           NaN           NaN      506417.0   
2338           NaN       62273.0           NaN           NaN       57819.0   
3463           NaN      322008.0           NaN           NaN      300956.0   
3464           NaN           NaN           NaN           NaN           NaN   
3465           NaN        7542.0           NaN           NaN        6979.0   
3466           NaN        8036.0           NaN           NaN        7428.0   
3467           NaN        9990.0           NaN           NaN        9269.0   
3468           NaN       10276.0           NaN           NaN        9502.0   
3469           NaN       10483.0           NaN           NaN        9710.0   
3470           NaN        9319.0           NaN           NaN        8632.0   

     E_A_HOS_C1_48 E_A_HOS_C1_7099  Line 19 Subtotal  \
2337      439158.0            17.0          542984.0   
2338        5749.0            53.0           62273.0   
3463      265783.0            59.0          322008.0   
3464           NaN            63.0               NaN   
3465           NaN            75.0            7542.0   
3466           NaN            80.0            8036.0   
3467           NaN           100.0            9990.0   
3468           NaN           103.0           10276.0   
3469           NaN           105.0           10483.0   
3470           NaN           116.0            9319.0   

      Reconstructed IPPS payment (pre HAC penalty)  Reconstructed HAC penalty  \
2337                                      678200.0        

In [7]:
del nmrc_df
del alpha_df
del df
del e_df
del s_df
del final_df

main_df2.to_pickle('~/GitHub/HCRIS-databuilder/Filtered_PUF_data/FilteredEngineeredPUF_p5.pkl', protocol=5)